# Worksheet 08

Name: Haokun Wu
UID: U18377013

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [3]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.396207446771601, 3.1485032374514943, 5.037613712460368, 6.005984841848162, 5.595632417419438, 6.069185361625392, 6.201117980342201, 4.042670827914843, 5.7329791542707715, 5.1898889928732235]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal(8, 1, 10).tolist()
print(c2)


[7.954014907903752, 9.325438679551066, 8.102790213599588, 7.308448657056504, 8.442478776761392, 9.064731701975022, 7.202710630415226, 8.022776244634088, 8.342712942228315, 7.92616805074897]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0 and c1:  
        p1 = c1.pop() 
        data.append(p1)
    elif coin_output == 1 and c2: 
        p2 = c2.pop()
        data.append(p2)
print(data)


[5.1898889928732235, 7.92616805074897, 5.7329791542707715, 4.042670827914843, 6.201117980342201, 6.069185361625392, 8.342712942228315, 8.022776244634088, 5.595632417419438, 7.202710630415226]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

GMM is defined by the parameters (µ, Σ, π),

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(n_clusters=2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]

prob_c = [len(c1) / (len(c1) + len(c2)), len(c2) / (len(c1) + len(c2))]
mean = [sum(c1) / len(c1), sum(c2) / len(c2)]
var = [sum(map(lambda x: (x - mean[0])**2, c1)) / len(c1), sum(map(lambda x: (x - mean[1])**2, c2)) / len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))


P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 5.471912455740978,  mean_2 = 7.87359196700665
var_1 = 0.5157011559217574,  var_2 = 0.17379411261313032


/Users/kyle/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The estimated means are relatively close to the true values, but the variances from the K-means initialization are underestimated. This isn't surprising, as K-means clustering tends to form spherical clusters, which might not capture the full spread of the data.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x_list = [] # P(X_i) for each data point

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))  # Use square root of variance to get std deviation
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]
    prob_x_list.append(prob_x)

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()



point =  5.1898889928732235
probability of observing that point if it came from cluster 0 =  0.5143042617733339
probability of observing that point if it came from cluster 1 =  9.594425291343615e-10
point =  7.92616805074897
probability of observing that point if it came from cluster 0 =  0.0016159920843926323
probability of observing that point if it came from cluster 1 =  0.9493770899851134
point =  5.7329791542707715
probability of observing that point if it came from cluster 0 =  0.5200109820725884
probability of observing that point if it came from cluster 1 =  1.8014773737778776e-06
point =  4.042670827914843
probability of observing that point if it came from cluster 0 =  0.07666095442879262
probability of observing that point if it came from cluster 1 =  4.405574581683192e-19
point =  6.201117980342201
probability of observing that point if it came from cluster 0 =  0.331749267884466
probability of observing that point if it came from cluster 1 =  0.0003061726722995627
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]

mean = [
    sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x),
    sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)
]

var = [
    sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x),
    sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x)
]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))


P(C_1) = 0.615233870435623,  P(C_2) = 0.38476612956437684
mean_1 = 5.5155224417058255,  mean_2 = 7.89894901716791
var_1 = 0.5782023632028392,  var_2 = 0.16320824339288234


The updated parameters, derived from the E-step of the EM algorithm, are indeed a refinement of the initial estimates from K-means. The slight changes in probabilities, means, and variances better represent the Gaussian components underlying the mixed data. The adjustments, while not drastic, bring the parameters closer to the true nature of the data, even if they're not yet on the true values.



g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
from scipy.stats import norm

updated_prob_c0_x = [] # Updated P(C_0 | X_i)
updated_prob_c1_x = [] # Updated P(C_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(2):
        # P(X_i | C_j) for the updated parameters
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    updated_prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    updated_prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

# Let's print and compare
for i in range(len(data)):
    print(f"Data point {i+1}: {data[i]}")
    print(f"Initial P(C_1 | X_i): {prob_c0_x[i]}, Updated: {updated_prob_c0_x[i]}")
    print(f"Initial P(C_2 | X_i): {prob_c1_x[i]}, Updated: {updated_prob_c1_x[i]}")
    print()


Data point 1: 5.1898889928732235
Initial P(C_1 | X_i): 0.9999999987563231, Updated: 0.9999999997780985
Initial P(C_2 | X_i): 1.2436769428184312e-09, Updated: 2.2190150249055983e-10

Data point 2: 7.92616805074897
Initial P(C_1 | X_i): 0.002546738160244521, Updated: 0.0055629247866357525
Initial P(C_2 | X_i): 0.9974532618397556, Updated: 0.9944370752133643

Data point 3: 5.7329791542707715
Initial P(C_1 | X_i): 0.9999976904677332, Updated: 0.9999992974441932
Initial P(C_2 | X_i): 2.3095322667067406e-06, Updated: 7.025558067539655e-07

Data point 4: 4.042670827914843
Initial P(C_1 | X_i): 1.0, Updated: 1.0
Initial P(C_2 | X_i): 3.831219873280132e-18, Updated: 1.2587187239488694e-19

Data point 5: 6.201117980342201
Initial P(C_1 | X_i): 0.9993851090986869, Updated: 0.999741819186455
Initial P(C_2 | X_i): 0.0006148909013130486, Updated: 0.0002581808135449229

Data point 6: 6.069185361625392
Initial P(C_1 | X_i): 0.9998612687559991, Updated: 0.9999461243469366
Initial P(C_2 | X_i): 0.000138

The updates to the parameters have generally caused the posterior probabilities \( P(C_j | X_i) \) to increase slightly for \( C_1 \) and decrease slightly for \( C_2 \). 

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
assignments = []

for i in range(len(data)):
    if updated_prob_c0_x[i] > updated_prob_c1_x[i]:
        assignments.append(0)
    else:
        assignments.append(1)

for i, point in enumerate(data):
    print(f"Data point {i+1}: {point}, assigned to cluster {assignments[i]}")


Data point 1: 5.1898889928732235, assigned to cluster 0
Data point 2: 7.92616805074897, assigned to cluster 1
Data point 3: 5.7329791542707715, assigned to cluster 0
Data point 4: 4.042670827914843, assigned to cluster 0
Data point 5: 6.201117980342201, assigned to cluster 0
Data point 6: 6.069185361625392, assigned to cluster 0
Data point 7: 8.342712942228315, assigned to cluster 1
Data point 8: 8.022776244634088, assigned to cluster 1
Data point 9: 5.595632417419438, assigned to cluster 0
Data point 10: 7.202710630415226, assigned to cluster 1


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      no      |
| A  C |      yes     |
| A  D |      yes     |
| A  E |      yes     |
| B  C |      yes     |
| B  D |      yes     |
| B  E |      yes     |
| C  D |      yes     |
| C  E |      yes     |
| D  E |      no      |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

N*(N-1)/2

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

3 + 1 + 6 = 10

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

36 - 10 = 26

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

For each cluster in P, we quickly filter and isolate its data points. We then calculate the number of unique point pairs within each cluster, treating these as "agreements" since both clustering methods group them together. The total disagreements are then deduced by subtracting the total agreements from the overall possible unique pairs in the dataset. 